In [ ]:
# Threading
from threading import Thread
import time

def square_numbers():
    for i in range(100):
        i * i
        
if __name__ == "__main__":
    threads = []
    num_threads = 10
    
# Create threads
for i in range(num_threads):
    thread = Thread(target=square_numbers)
    threads.append(thread)
    
# Start threads
for thread in threads:
    thread.start()

# Join threads: wait for them to complete
for thread in threads:
    thread.join()
    
print('end main')

In [ ]:
database_value = 0
def increase():
    global database_value
    
    local_copy = database_value #local_copy = 0 after this line execution
    
    # processing
    local_copy += 1 # local_copy = 1 after this line execution
    time.sleep(0.1) # thread will switch to thread #2, and at this moment database_value is still 0 since next
                    # line has not been executed yet. Thread #2 do the same thing, and local_copy is 1
    database_value = local_copy #database_value = 1 after this line execution
    
if __name__ == "__main__":
    
    print('start value', database_value)
    
    thread1 = Thread(target=increase)
    thread2 = Thread(target=increase)
    
    thread1.start()
    thread2.start()
    
    thread1.join()
    thread2.join()
    
    # We got end value of 1 because thread 1 and thread 2 are both changing the same variable at the same time
    print('end value', database_value) # database_value = 1
    
    print('end main')

In [ ]:
# How do we prevent the problem in the cell above? We use Lock
from threading import Lock
database_value = 0
def increase(lock):
    global database_value
    
    lock.acquire()
    local_copy = database_value #local_copy = 0 after this line execution
    
    # processing
    local_copy += 1 # local_copy = 1 after this line execution
    time.sleep(0.1) # thread will switch to thread #2, and at this moment database_value is still 0 since next
                    # line has not been executed yet. Thread #2 do the same thing, and local_copy is 1
    database_value = local_copy #database_value = 1 after this line execution
    lock.release() # Everytime we acquire a lock, we have to release it
    
if __name__ == "__main__":
    
    lock = Lock()
    print('start value', database_value)
    
    thread1 = Thread(target=increase, args=(lock,)) # since args is tuple, and we only have one element 'lock',
                                                    # we need to add "," to tell it is tuple
    thread2 = Thread(target=increase, args=(lock,))
    
    thread1.start()
    thread2.start()
    
    thread1.join()
    thread2.join()
    
    # We got end value of 1 because thread 1 and thread 2 are both changing the same variable at the same time
    print('end value', database_value) # database_value = 1
    
    print('end main')

In [ ]:
# Instead of using "lock.acquire()" and "lock.release()", we can use "with lock:" to achieve same effect
from threading import Lock
database_value = 0
def increase(lock):
    global database_value
    
    with lock:
        local_copy = database_value #local_copy = 0 after this line execution

        # processing
        local_copy += 1 # local_copy = 1 after this line execution
        time.sleep(0.1) # thread will switch to thread #2, and at this moment database_value is still 0 since next
                        # line has not been executed yet. Thread #2 do the same thing, and local_copy is 1
        database_value = local_copy #database_value = 1 after this line execution
    
if __name__ == "__main__":
    
    lock = Lock()
    print('start value', database_value)
    
    thread1 = Thread(target=increase, args=(lock,)) # since args is tuple, and we only have one element 'lock',
                                                    # we need to add "," to tell it is tuple
    thread2 = Thread(target=increase, args=(lock,))
    
    thread1.start()
    thread2.start()
    
    thread1.join()
    thread2.join()
    
    # We got end value of 1 because thread 1 and thread 2 are both changing the same variable at the same time
    print('end value', database_value) # database_value = 1
    
    print('end main')

In [ ]:
# queue: FIFO
from threading import current_thread
from queue import Queue

if __name__ == "__main__":
    
    q = Queue()
    
    q.put(1) # 1
    q.put(2) # 2 1
    q.put(3) # 3 2 1 --->
    
    first = q.get()
    print(first)
    
    # Every time when we use "q.get", we have to add "task_done()" to finish it
    q.task_done()
    
    print('end main')

In [2]:
from threading import Thread, Lock, current_thread
from queue import Queue
import time

def worker(q, lock):
    while True:
        value = q.get()
        
        #processing...
        with lock:
            print(f'in {current_thread().name} got {value}')
        q.task_done()
        
if __name__ == "__main__":
    
    q = Queue()
    lock = Lock()
    num_threads = 10
    
    for i in range(num_threads):
        thread = Thread(target=worker, args=(q, lock))
        thread.daemon = True
        thread.start()
        
    for i in range(1, 21):
        q.put(i)
        
    q.join()
    
    print('end main')

in Thread-14 got 1
in Thread-14 got 3
in Thread-14 got 4
in Thread-14 got 5
in Thread-14 got 6
in Thread-14 got 7
in Thread-14 got 8
in Thread-14 got 9
in Thread-14 got 10
in Thread-14 got 11
in Thread-14 got 12
in Thread-14 got 13
in Thread-14 got 14
in Thread-14 got 15
in Thread-14 got 16
in Thread-14 got 17
in Thread-15 got 18
in Thread-20 got 19
in Thread-21 got 20
in Thread-16 got 2
end main
